In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# ✅ 데이터 로드
body_type_df = pd.read_csv("data/merged_body_type.csv", encoding="utf-8")
food_nutrition_df = pd.read_csv("data/food_nutrition_data.csv", encoding="utf-8")

# ✅ 체형 데이터 전처리
body_type_df["성별"] = body_type_df["성별"].map({"남성": 0, "여성": 1})

# 필요식단 키워드 추출
body_type_df["저탄수화물"] = body_type_df["필요식단"].str.contains("저탄수화물").astype(int)
body_type_df["고단백"] = body_type_df["필요식단"].str.contains("고단백").astype(int)
body_type_df["고지방"] = body_type_df["필요식단"].str.contains("고지방").astype(int)

# ✅ MinMaxScaler 적용 (🔹 영양소 정규화 먼저 수행)
scaler = MinMaxScaler()
nutrition_cols = ["칼로리", "탄수화물", "단백질", "지방"]

# 🔥 원본 데이터 백업
merged_df["칼로리_원본"] = merged_df["칼로리"]
merged_df["탄수화물_원본"] = merged_df["탄수화물"]
merged_df["단백질_원본"] = merged_df["단백질"]
merged_df["지방_원본"] = merged_df["지방"]

# 🔥 MinMaxScaler 적용 (정규화)
merged_df[nutrition_cols] = scaler.fit_transform(merged_df[nutrition_cols])

# ✅ key 컬럼 추가 (🔥 중요!!)
body_type_df["key"] = 1
food_nutrition_df["key"] = 1  

# ✅ Cross Join 수행 (체형 × 성별 × 음식)
expanded_food_nutrition_df = pd.merge(
    food_nutrition_df, 
    body_type_df[["체형", "성별", "key"]], 
    on="key"
).drop(columns=["key"])

# ✅ 병합 (체형 + 성별 기준)
merged_df = pd.merge(body_type_df, expanded_food_nutrition_df, on=["체형", "성별"], how="left")

# ✅ 병합 후 NaN 값 확인 (🔍 디버깅)
print("🚀 병합 후 NaN 개수:\n", merged_df[nutrition_cols].isna().sum())

# ✅ 최종 데이터 저장
merged_df.to_csv("final_dataset.csv", index=False, encoding="utf-8-sig")
print("✅ final_dataset.csv 저장 완료!")

🚀 병합 후 NaN 개수:
 칼로리     0
탄수화물    0
단백질     0
지방      0
dtype: int64
✅ final_dataset.csv 저장 완료!
